In [6]:
#coding:utf-8
import pandas as pd
import numpy as np
import time
import datetime
import gc
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [7]:
#数据预处理 (pre_process 函数)，从数据中删除不必要的列（instance_id 和 day）返回处理后的数据和要删除的列名列表。
def pre_process(data):
    
    cols = data.columns.tolist()
    keys = ['instance_id', 'day']
    for k in keys:
        cols.remove(k)

    return data, cols

In [8]:
#滚动窗口特征生成 (dorollWin 函数):将context_timestamp转换为字符串类型，并创建基于时间的各种特征，
#计算用户在10分钟窗口内的事件数（点击次数）(user_count_10_bf 和 user_count_10_af)，类似的计算应用于用户-商店和用户-商品组合。

#将 'context_timestamp' 列的时间戳转换为字符串类型，并创建一个名为 'context_timestamp_str' 的新列，其中包含时间戳的字符串表示
#按用户ID分组，并将每个用户的 'context_timestamp_str' 连接成一个由分号分隔的字符串。然后，将这些信息合并回原始数据中，创建一个名为 'user_time_join' 的新列
#用户-商店和用户-商品组合进行类似

def dorollWin(data):

    data['context_timestamp_str'] = data['context_timestamp'].astype(str)
    user_time_join = data.groupby('user_id')['context_timestamp_str'].agg(lambda x:';'.join(x)).reset_index()
    user_time_join.rename(columns={'context_timestamp_str':'user_time_join'},inplace = True)

    data = pd.merge(data,user_time_join,on=['user_id'],how='left')
    user_shop_time_join = data.groupby(['user_id','shop_id'])['context_timestamp_str'].agg(lambda x:';'.join(x)).reset_index()
    user_shop_time_join.rename(columns={'context_timestamp_str':'user_shop_time_join'},inplace = True)

    data = pd.merge(data,user_shop_time_join,on=['user_id','shop_id'],how='left')
    user_item_time_join = data.groupby(['user_id','item_id'])['context_timestamp_str'].agg(lambda x:';'.join(x)).reset_index()
    user_item_time_join.rename(columns={'context_timestamp_str':'user_item_time_join'},inplace = True)
    
    data = pd.merge(data,user_item_time_join,on=['user_id','item_id'],how='left')
    data['index_']=data.index
    del user_time_join,user_shop_time_join,user_item_time_join
    
    nowtime=data.context_timestamp.values
    user_time=data.user_time_join.values
    user_shop_time=data.user_shop_time_join.values
    user_item_time=data.user_item_time_join.values
    
    data_len=data.shape[0]
    user_time_10_bf=np.zeros(data_len)
    user_time_10_af=np.zeros(data_len)
    user_shop_time_10_bf=np.zeros(data_len)
    user_shop_time_10_af=np.zeros(data_len)
    user_item_time_10_bf=np.zeros(data_len)
    user_item_time_10_af=np.zeros(data_len)
    a=time.time()
    for i in range(data_len):
        df1=nowtime[i]
        df2=user_time[i].split(';')
        df2_len=len(df2)
        for j in range(df2_len):
            if ((int(df2[j])-df1)<600) & ((int(df2[j])-df1)>0):
                user_time_10_bf[i]+=1
            if ((int(df2[j])-df1)>-600) & ((int(df2[j])-df1)<0):
                user_time_10_af[i]+=1
        
        df3=user_shop_time[i].split(';')
        df3_len=len(df3)
        for j in range(df3_len):
            if ((int(df3[j])-df1)<600) & ((int(df3[j])-df1)>0):
                user_shop_time_10_bf[i]+=1
            if ((int(df3[j])-df1)>-600) & ((int(df3[j])-df1)<0):
                user_shop_time_10_af[i]+=1
                
        df4=user_item_time[i].split(';')
        df4_len=len(df4)
        for j in range(df4_len):
            if ((int(df4[j])-df1)<600) & ((int(df4[j])-df1)>0):
                user_item_time_10_bf[i]+=1
            if ((int(df4[j])-df1)>-600) & ((int(df4[j])-df1)<0):
                user_item_time_10_af[i]+=1
                
    print(time.time()-a)
    
    data['user_count_10_bf']=user_time_10_bf
    data['user_count_10_af']=user_time_10_af
    data['user_shop_count_10_bf']=user_shop_time_10_bf
    data['user_shop_count_10_af']=user_shop_time_10_af
    data['user_item_count_10_bf']=user_item_time_10_bf
    data['user_item_count_10_af']=user_item_time_10_af

    drops = ['context_timestamp_str', 'user_time_join', 'user_shop_time_join',
       'user_item_time_join', 'index_']
    data = data.drop(drops, axis=1)
    
    return data

In [9]:
#大小（size）特征生成 (doSize 函数):计算每个商店在给定日期内唯一的商品数量 (shop_item_unique_day)
#计算每个用户在给定日期内的查询次数 (user_id_query_day)
#将 'minute' 特征分成不同的时间间隔（10、15、30、45分钟）
#计算用户在这些时间间隔内的点击次数 (min10_user_click、min15_user_click、min30_user_click、min45_user_click)
def doSize(data):
  

  #按用户、用户-商店、用户-商品分组并连接时间戳字符串
    add = pd.DataFrame(data.groupby(["shop_id", "day"]).item_id.nunique()).reset_index()
    add.columns = ["shop_id", "day", "shop_item_unique_day"]
    data = data.merge(add, on=["shop_id", "day"], how="left")

    user_query_day = data.groupby(['user_id', 'day']).size().reset_index().rename(columns={0: 'user_id_query_day'})
    data = pd.merge(data, user_query_day, how='left', on=['user_id', 'day'])
    
    data['min_10'] = data['minute'] // 10
    data['min_15'] = data['minute'] // 15
    data['min_30'] = data['minute'] // 30
    data['min_45'] = data['minute'] // 45
    
    # user 不同时间段点击次数，这一步通过循环遍历数据集，在每个数据点处计算用户在10分钟窗口内的事件数（点击次数），包括用户、用户-商店和用户-商品的组合
    min10_user_click = data.groupby(['user_id', 'day', 'hour', 'min_10']).size().reset_index().rename(columns={0:'min10_user_click'})
    min15_user_click = data.groupby(['user_id', 'day', 'hour', 'min_15']).size().reset_index().rename(columns={0:'min15_user_click'})
    min30_user_click = data.groupby(['user_id', 'day', 'hour', 'min_30']).size().reset_index().rename(columns={0:'min30_user_click'})
    min45_user_click = data.groupby(['user_id', 'day', 'hour', 'min_45']).size().reset_index().rename(columns={0:'min45_user_click'})

    data = pd.merge(data, min10_user_click, 'left', on=['user_id', 'day', 'hour', 'min_10'])
    data = pd.merge(data, min15_user_click, 'left', on=['user_id', 'day', 'hour', 'min_15'])
    data = pd.merge(data, min30_user_click, 'left', on=['user_id', 'day', 'hour', 'min_30'])
    data = pd.merge(data, min45_user_click, 'left', on=['user_id', 'day', 'hour', 'min_45'])
    
    del data['min_10']
    del data['min_15']
    del data['min_30']
    del data['min_45']

    return data

In [10]:
path = '/Users/apple/Desktop/data/'

train = pd.read_csv(path+'train_all.csv')
test = pd.read_csv(path+'test_all.csv')

data = pd.concat([train, test])
print('初始维度:', data.shape)
    
data, cols = pre_process(data)
print('pre_process:', data.shape)
    
##################################
data = dorollWin(data)
print('dorollWin:', data.shape)

data = doSize(data)
print('doSize:', data.shape)
##################################
    
data = data.drop(cols, axis=1)

# 得到全部训练集
print('经过处理后,全部训练集最终维度:', data.shape)
data.to_csv(path+'all_03.csv', index=False)

    
# 得到7号训练集
data = data.loc[data.day == 7]
print('经过处理后,7号数据集最终维度::',data.shape)
print(data.columns.tolist())
data.to_csv(path+'day7_03.csv', index=False)

初始维度: (99085, 35)
pre_process: (99085, 35)
0.544348955154419
dorollWin: (99085, 41)
doSize: (99085, 47)
经过处理后,全部训练集最终维度: (99085, 14)
经过处理后,7号数据集最终维度:: (10184, 13)
['instance_id', 'user_count_10_bf', 'user_count_10_af', 'user_shop_count_10_bf', 'user_shop_count_10_af', 'user_item_count_10_bf', 'user_item_count_10_af', 'shop_item_unique_day', 'user_id_query_day', 'min10_user_click', 'min15_user_click', 'min30_user_click', 'min45_user_click']
